<a href="https://colab.research.google.com/github/jjome/team1_visual/blob/main/cleansing_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is maded to cleansing for landing-club data
 : for team-project in the cource of 8th fintech in SNU

 - The name of train data : "lending_club_2020_train.csv"
 - The name of test data : "lending_club_2020_test.csv"


# Loading data file where is the format


In [39]:
from google.colab import drive   ## google drive mount
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
## include the libraries
import numpy as np
import pandas as pd
import sys, os

In [41]:
## loading the csv
dir_name = "/content/drive/Othercomputers/내 Mac/fintech/9.시각화/team_project/"
csv = dir_name+"orginal_data.csv"     # train data set to csv

df1 = pd.read_csv(csv, encoding='cp949')                          # data frame of csv, i.e. source file
df1.head()

,사용월,호선명,지하철역,유임승차인원,무임승차인원,유임하차인원,무임하차인원,작업일자
0,202403,1호선,종각,964737,152686,946524,142688,20240403
1,202403,1호선,서울역,1447754,227062,1393734,217639,20240403
2,202403,1호선,동묘앞,169654,183569,179509,185429,20240403
3,202403,1호선,종로5가,507635,270416,495448,260685,20240403
4,202403,1호선,제기동,247757,284893,240496,302034,20240403


In [42]:
Num = len(df1)                 # the number of lines to csv.
print(f"The total Number of columns of data are {Num}")

The total Number of columns of data are 67261


# Cleansing

In [43]:
feat = '작업일자'
df1.drop(columns = [feat], axis = 0, inplace = True)
print(df1.head())

      사용월  호선명  지하철역   유임승차인원  무임승차인원   유임하차인원  무임하차인원
0  202403  1호선    종각   964737  152686   946524  142688
1  202403  1호선   서울역  1447754  227062  1393734  217639
2  202403  1호선   동묘앞   169654  183569   179509  185429
3  202403  1호선  종로5가   507635  270416   495448  260685
4  202403  1호선   제기동   247757  284893   240496  302034


In [44]:
# 날짜형 변수의 경우, 'year'과 'month' feature로 나눠서 저장.
feat = '사용월'
df1[feat] = pd.to_datetime(df1[feat], format='%Y%m')
df1['year'] = df1[feat].dt.year.astype(int)
df1['month'] = df1[feat].dt.month.astype(int)

df1.drop(columns = [feat], axis = 0, inplace = True)
print(df1.head())

   호선명  지하철역   유임승차인원  무임승차인원   유임하차인원  무임하차인원  year  month
0  1호선    종각   964737  152686   946524  142688  2024      3
1  1호선   서울역  1447754  227062  1393734  217639  2024      3
2  1호선   동묘앞   169654  183569   179509  185429  2024      3
3  1호선  종로5가   507635  270416   495448  260685  2024      3
4  1호선   제기동   247757  284893   240496  302034  2024      3


# 역 정보

In [45]:
csv = dir_name+"data_loc.csv"     # train data set to csv

df2 = pd.read_csv(csv)                          # data frame of csv, i.e. source file

df2.rename(columns={'lineNm': '호선명', 'stnKrNm': '지하철역'}, inplace=True)
df2.head()

,outStnNum,지하철역,호선명,convX,convY
0,4128,삼성중앙,9호선2~3단계,127.053282,37.513011
1,4124,사평,9호선,127.015259,37.504206
2,4121,구반포,9호선,126.987332,37.501364
3,4119,흑석(중앙대입구),9호선,126.963708,37.508770
4,4116,샛강,9호선,126.928422,37.517274


In [46]:
# df2.sort_values(by=['지하철역', '호선명'])
# df2 = df2.groupby('지하철역').agg({
#     'convX': 'mean',
#     'convY': 'mean',
#     '호선명': 'first'
# }).reset_index().sort_values(by=['호선명', '지하철역'])
# df2.head()

In [58]:
df = pd.merge(df1, df2[['지하철역', '호선명', 'convX', 'convY']], on=['지하철역', '호선명'], how='left')
df.head()

,호선명,지하철역,유임승차인원,무임승차인원,유임하차인원,무임하차인원,year,month,convX,convY
0,1호선,종각,964737,152686,946524,142688,2024,3,126.982923,37.570161
1,1호선,서울역,1447754,227062,1393734,217639,2024,3,126.972135,37.556228
2,1호선,동묘앞,169654,183569,179509,185429,2024,3,127.016480,37.573197
3,1호선,종로5가,507635,270416,495448,260685,2024,3,127.001849,37.570926
4,1호선,제기동,247757,284893,240496,302034,2024,3,127.034893,37.578103


In [50]:
df.sort_values(by=['year','month', '지하철역', '호선명'], ascending=[False, False, True, True], inplace=True)
df.head()

,호선명,지하철역,유임승차인원,무임승차인원,유임하차인원,무임하차인원,year,month,convX,convY
581,우이신설선,4.19민주묘지,83280,39866,81586,35738,2024,3,127.013684,37.649502
400,경원선,가능,136796,61869,130217,61140,2024,3,127.044213,37.748577
85,3호선,가락시장,187801,84075,182752,81474,2024,3,127.117757,37.492245
267,8호선,가락시장,168173,68687,190267,69611,2024,3,127.118398,37.492888
251,7호선,가산디지털단지,1078509,104903,1078624,99644,2024,3,126.882656,37.480338


In [59]:
df.groupby(['year', 'month', '지하철역']).agg({
    '호선명': 'first',  # 호선명은 그룹의 첫 번째 값을 사용
    '유임승차인원': 'sum',  # 유임승차인원 합계
    '무임승차인원': 'sum',  # 무임승차인원 합계
    '유임하차인원': 'sum',  # 유임하차인원 합계
    '무임하차인원': 'sum',  # 무임하차인원 합계
    'convX': 'mean',
    'convY': 'mean'
}, inplace=True).reset_index().sort_values(by=['year','month', '호선명', '지하철역'], ascending=[False, False, True, True])
df.reset_index(drop=True, inplace=True)
df.head()

,호선명,지하철역,유임승차인원,무임승차인원,유임하차인원,무임하차인원,year,month,convX,convY
0,1호선,종각,964737,152686,946524,142688,2024,3,126.982923,37.570161
1,1호선,서울역,1447754,227062,1393734,217639,2024,3,126.972135,37.556228
2,1호선,동묘앞,169654,183569,179509,185429,2024,3,127.016480,37.573197
3,1호선,종로5가,507635,270416,495448,260685,2024,3,127.001849,37.570926
4,1호선,제기동,247757,284893,240496,302034,2024,3,127.034893,37.578103


# Save

In [60]:
# Save filename
filename = dir_name+"data_cs.csv"  # train data set to csv

# Save to csv
df.to_csv(filename, index=False, encoding='utf-8-sig')

# 이름이 달라 좌표가 누락된역들을 체크하는 코드임
필요에 따라 수작업으로 수정해야할듯

In [57]:
feat = 'convX'
missing = df.index[df[feat].isna()].tolist()
df_tmp = df.loc[missing]
lst = df_tmp['지하철역'].unique()
print(lst)
print(len(lst))
print(df_tmp.head(20))

['가좌' '강매' '검암' '계양' '곡산' '공덕' '공항화물청사' '금릉' '금촌' '김포공항' '능곡' '디지털미디어시티'
 '마곡나루(서울식물원)' '문산' '백마' '서강대' '서울역' '성남' '수색' '신촌' '야당' '영종' '운서' '운정'
 '운천' '월롱' '인천공항1터미널' '인천공항2터미널' '일산' '임진강' '자양(뚝섬한강공원)' '청라국제도시' '청산'
 '탄현' '파주' '평택지제' '풍산' '한국항공대' '행신' '홍대입구' '효창공원앞' '상봉(시외버스터미널)' '화전'
 '낙성대' '동대문역사문화공원' '마곡나루' '용마산' '봉은사' '삼성중앙' '선정릉' '언주' '종합운동장' '인천국제공항'
 '신천' '강변' '경복궁' '고려대' '공릉' '광나루' '광화문' '광흥창' '교대' '구의' '군자' '굽은다리'
 '남부터미널' '남한산성입구' '대림' '대흥' '동두천 중앙' '동작' '몽촌토성' '미아' '삼성' '상월곡' '새절'
 '서울대입구' '성신여대입구' '숙대입구' '숭실대입구' '신정' '신창' '쌍용동' '아차산' '안암' '양재' '어린이대공원'
 '오목교' '온수' '올림픽공원' '왕십리' '용두' '월곡' '월드컵경기장' '이촌' '잠실' '증산' '천호' '청량리(지상)'
 '청량리(지하)' '총신대입구' '충정로' '한성대입구' '화랑대' '회현']
105
          호선명         지하철역  유임승차인원  무임승차인원  유임하차인원  무임하차인원  year  month  \
425       경의선           가좌  139605   48561  129166   48601  2024      3   
423       경의선           강매   79315   20386   66786   19267  2024      3   
424       경의선           검암       3       0       0       0  2024      3   
478  